In [1]:
import sj_trading
from sj_trading.quote import QuoteManager
import polars as pl
import polars_talib as plta

In [2]:
api = sj_trading.get_api()

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [3]:
quote_manager = QuoteManager(api)

In [5]:
quote_manager.subscribe_stk_tick(["2330", "2890", "2317"], recover=True)

Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2330 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2890 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2317 | Event: Subscribe or Unsubscribe ok


In [6]:
df_ticks = quote_manager.get_df_stk()

In [7]:
df_ticks

datetime,code,price,volume,tick_type
datetime[μs],str,f64,i64,i8
2024-10-09 09:00:06.187239,"""2330""",1030.0,4932,1
2024-10-09 09:00:06.271956,"""2330""",1025.0,1,2
2024-10-09 09:00:06.367591,"""2330""",1030.0,158,1
2024-10-09 09:00:06.380447,"""2330""",1030.0,4,1
2024-10-09 09:00:06.381299,"""2330""",1025.0,1,2
…,…,…,…,…
2024-10-09 13:24:57.021834,"""2317""",198.0,1,2
2024-10-09 13:24:57.669549,"""2317""",198.5,1,1
2024-10-09 13:24:58.217345,"""2317""",198.5,5,1


In [13]:
quote_manager.get_df_stk_kbar("5m", [
    pl.col("close").ta.ema(5).over("code").fill_nan(None).alias("ema5"),
    plta.macd(pl.col("close"), 12, 26, 9).over("code").struct.field("macd").fill_nan(None),
])

datetime,code,open,high,low,close,volume,ema5,macd
datetime[μs],str,f64,f64,f64,f64,i64,f64,f64
2024-10-09 09:00:00,"""2330""",1030.0,1030.0,1025.0,1025.0,6260,null,null
2024-10-09 09:05:00,"""2330""",1025.0,1030.0,1025.0,1025.0,738,null,null
2024-10-09 09:10:00,"""2330""",1025.0,1030.0,1025.0,1025.0,552,null,null
2024-10-09 09:15:00,"""2330""",1030.0,1030.0,1025.0,1025.0,438,null,null
2024-10-09 09:20:00,"""2330""",1025.0,1030.0,1025.0,1025.0,254,1025.0,null
…,…,…,…,…,…,…,…,…
2024-10-09 13:05:00,"""2317""",198.5,198.5,198.0,198.5,384,198.408477,-0.010153
2024-10-09 13:10:00,"""2317""",198.5,198.5,198.0,198.0,1146,198.272318,-0.042684
2024-10-09 13:15:00,"""2317""",198.5,199.0,198.0,199.0,1588,198.514879,0.012087


In [14]:
quote_manager.unsubscribe_all_stk_tick()

Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2317 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2330 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2890 | Event: Subscribe or Unsubscribe ok
